In [1]:
%matplotlib inline

import random

import pandas as pd
import numpy as np
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as sp

# import xgboost as xgb
from itertools import tee, islice
from ROOT import TFile, TCanvas, TPad, TPaveLabel, TPaveText, TTree, TH1F, TF1
from root_numpy import root2array, tree2array, array2tree, array2root
import sys
from ROOT import gROOT, AddressOf
from root_numpy import root2array, rec2array


from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score

Welcome to JupyROOT 6.22/02


In [2]:
import matplotlib.ticker as ticker
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn import tree



In [3]:
back = 'back.root'
signal = 'signal.root'
# out_dir = '/afs/cern.ch/user/s/sraj/public/plot'
treeName_back = "tagsDumper/trees/thq_125_13TeV_THQLeptonicTag"
treeName_signal = "tagsDumper/trees/tth_125_13TeV_THQLeptonicTag"
columns = ['dipho_pt','dipho_phi','dipho_eta','dipho_e','dipho_mass','dipho_leadPt','dipho_leadEt','dipho_leadEta','dipho_leadPhi','dipho_subleadEta','bjet1_pt',
          'bjet2_pt','bjet1_eta','bjet2_eta','jet1_pt','jet2_pt','jet1_eta','n_jets']

columns = [c.strip() for c in columns]
columns = (b.replace(" ", "_") for b in columns)
columns = list(b.replace("-", "_") for b in columns)

mc_arr = root2array(back, treeName_back, columns)
data_arr = root2array(signal, treeName_signal, columns)

signal = rec2array(data_arr)
backgr = rec2array(mc_arr)


In [4]:
# for sklearn data is usually organised
# into one 2D array of shape (n_samples x n_features)
# containing all the data and one array of categories
# of length n_samples
X = np.concatenate((signal, backgr))
y = np.concatenate((np.ones(signal.shape[0]),
                    np.zeros(backgr.shape[0])))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state =1)

In [ ]:
X_dev,X_eval, y_dev,y_eval = train_test_split(X, y,
                                              test_size=0.33, random_state=42)
X_train,X_test, y_train,y_test = train_test_split(X_dev, y_dev,
                                                  test_size=0.33, random_state=410)

In [6]:
clf = AdaBoostClassifier(base_estimator= tree.DecisionTreeClassifier(max_depth=2),
                          n_estimators=100, random_state=1,learning_rate=1)
# Fit the decision tree classifier
clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=2,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [ ]:
## create and run the SVC model with an rbf kernel
## some tuning has been done to get the gamma and C values
clf = SVC(C =2.0, kernel = 'rbf', tol = 0.001, gamma = 0.05)
clf.fit(X_train, y_train)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
##DNN
#define model for training
clf = Sequential()
clf.add(Dense(300, activation= 'relu', input_shape = (18,)))
# clf.add(Dense(300, activation= 'relu'))
# clf.add(Dense(300, activation= 'relu'))
# clf.add(Dense(300,activation= 'relu'))
# clf.add(Dense(300,activation= 'relu'))
clf.add(Dense(1, activation= 'sigmoid'))

clf.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
# clf.summary()


In [ ]:
clf.fit(X_train, y_train, epochs = 30, batch_size= 900, validation_split = 0.33)

In [ ]:
# Create classifier object and train
# Add code here to include other claassifiers (MLP, BDT,...)
clf = MLPClassifier(hidden_layer_sizes=(300,), activation='relu',max_iter=2000, random_state=0)
clf.fit(X_train, y_train)


In [ ]:
X_bkg_test = X_test[y_test==0]
X_sig_test = X_test[y_test==1]
y_bkg_test = y_test[y_test==0]
y_sig_test = y_test[y_test==1]
# if hasattr(clf, "predict_proba"):
#     tCut = 0.
#     y_bkg_pred = (clf.decision_function(X_bkg_test) >= tCut).astype(bool)
#     y_sig_pred = (clf.decision_function(X_sig_test) >= tCut).astype(bool)
# else:
pMin = 0.9
y_bkg_pred = (clf.predict_proba(X_bkg_test) >= pMin).astype(bool)
y_sig_pred = (clf.predict_proba(X_sig_test) >= pMin).astype(bool)
#for DNN there is no need of index[:,1] in "y_bkg_pred = (clf.predict_proba(X_bkg_test)[:,1]  >= pMin).astype(bool)
#" and "y_sig_pred = (clf.predict_proba(X_sig_test)[:,1]  >= pMin).astype(bool)
#" as index 1 is out of bounds for axis 1 with size 1

In [ ]:
power = metrics.accuracy_score(y_sig_test, y_sig_pred)        # = = Prob(t >= tCut|sig)
print('power of test with respect to signal = ', power)

#  Add code here to obtain the background efficiency
# = size of test alpha = = Prob(t >= tCut|bkg)


In [ ]:
# make histogram of decision function
plt.figure()                                     # new window
matplotlib.rcParams.update({'font.size':14})     # set all font sizes
tTest = clf.predict_proba(X_test)
# if hasattr(clf, "decision_function"):
#     tTest = clf.decision_function(X_test)        # if available use decision_function
# else:
#     tTest = clf.predict_proba(X_test)[:,1]       # for e.g. MLP need to use predict_proba
tBkg = tTest[y_test==0]
tSig = tTest[y_test==1]
nBins = 20
tMin = np.floor(np.min(tTest))
tMax = np.ceil(np.max(tTest))
bins = np.linspace(tMin, tMax, nBins+1)
plt.xlabel('decision function $t$', labelpad=3)
plt.ylabel('$f(t)$', labelpad=3)
n, bins, patches = plt.hist(tSig, bins=bins, density=True, histtype='step',color = 'b', fill=True)
n, bins, patches = plt.hist(tBkg, bins=bins, density=True, histtype='step', fill=True, alpha=0.5)
plt.show()